pip install numpy

In [5]:
pip install music21

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install keras

Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install --upgrade keras

   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   - -------------------------------------- 0.0/1.1 MB 262.6 kB/s eta 0:00:05
   - -------------------------------------- 0.0/1.1 MB 262.6 kB/s eta 0:00:05
   - -------------------------------------- 0.0/1.1 MB 219.4 kB/s eta 0:00:05
   -- ------------------------------------- 0.1/1.1 MB 252.2 kB/s eta 0:00:05
   -- ------------------------------------- 0.1/1.1 MB 252.2 kB/s eta 0:00:05
   -- ------------------------------------- 0.1/1.1 MB 252.2 kB/s eta 0:00:05
   --- ------------------------------------ 0.1/1.1 MB 273.1 kB/s eta 0:00:04
   --- ------------------------------------ 0.1/1.1 MB 273.1 kB/s eta 0:00:04
   ---- ----------------------------------- 0.1/1.1 MB 257.8 kB/s eta 0:00:04
   ----- ---------------------------------- 0.1/1.1 MB 275.0 kB/s eta 0:00:04
   ----- ----

In [4]:
import glob
import numpy as np
from music21 import converter, instrument, note, chord, stream
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense, Activation, InputLayer
from tensorflow.keras.optimizers import Adam

def get_notes():
    notes = []
    for file in glob.glob("House_of_Fun.mid"):
        midi = converter.parse(file)
        notes_to_parse = None
        parts = instrument.partitionByInstrument(midi)
        if parts:  # file has instrument parts
            notes_to_parse = parts.parts[0].recurse()
        else:  # file has notes in a flat structure
            notes_to_parse = midi.flat.notes
        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))
    return notes

notes = get_notes()
n_vocab = len(set(notes))

sequence_length = 100
pitchnames = sorted(set(item for item in notes))
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

network_input = []
network_output = []

for i in range(len(notes) - sequence_length):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])

n_patterns = len(network_input)

network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
network_input = network_input / float(n_vocab)
network_output = to_categorical(network_output)

model = Sequential()
model.add(InputLayer(input_shape=(sequence_length, 1)))  # Use InputLayer to define the shape
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001))

# Summary of the model
model.summary()


C:\Users\User\anaconda3\Lib\site-packages\keras\src\layers\core\input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 100, 256)       │       264,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 100, 256)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 100, 256)       │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 100, 256)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 256)            │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 21)             │         5,397 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 21)             │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,320,213 (5.04 MB)

 Trainable params: 1,320,213 (5.04 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
model.fit(network_input, network_output, epochs=100, batch_size=64)

Epoch 1/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 27s 2s/step - loss: 2.9188
Epoch 2/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - loss: 2.6532
Epoch 3/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - loss: 2.5411
Epoch 4/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - loss: 2.5303
Epoch 5/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - loss: 2.5275
Epoch 6/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - loss: 2.5056
Epoch 7/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - loss: 2.5127
Epoch 8/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - loss: 2.5254
Epoch 9/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - loss: 2.5410
Epoch 10/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - loss: 2.5732
Epoch 11/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - loss: 2.5620
Epoch 12/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - loss: 2.5071
Epoch 13/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - loss: 2.5337
Epoch 14/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - loss: 2.5192
Epoch 15/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - loss: 2.5144
Epoch 16/100
7/7 ━━━━━━━━━━━━━━━━━

In [19]:
def generate_notes(model, network_input, int_to_note, n_vocab):
    start = np.random.randint(0, len(network_input) - 1)
    pattern = network_input[start]
    prediction_output = []

    for note_index in range(500):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = np.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)

        pattern = np.append(pattern, index)
        pattern = pattern[1:len(pattern)]

    return prediction_output

int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
prediction_output = generate_notes(model, network_input, int_to_note, n_vocab)


In [21]:
def create_midi(prediction_output):
    offset = 0
    output_notes = []

    for pattern in prediction_output:
        if ('.' in pattern) or pattern.isdigit():
            chord_notes = pattern.split('.')
            chord_notes = [int(i) for i in chord_notes]
            new_chord = chord.Chord(chord_notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            output_notes.append(new_note)
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='output.mid')

create_midi(prediction_output)
